In [6]:
import numpy as np
from scipy import optimize as opt, linalg as la
from scipy.stats import multivariate_normal as mvn
from matplotlib import pyplot as plt
plt.style.use('ggplot')

In [10]:
time_series_a = np.loadtxt('time_series_a.txt')
time_series_b = np.loadtxt('time_series_b.txt')
time_series_c = np.loadtxt('time_series_c.txt')

# Problem 1

In [20]:
def arma_likelihood(time_series, phis=np.array([]), 
                    thetas=np.array([]), mu=0., sigma=1.):
    """
    Return the log-likelihood of the ARMA model parameters, given the time
    series.
    
    Parameters
    ----------
    time_series : ndarray of shape (n,1)
        The time series in question, z_t
    phis : ndarray of shape (p,)
        The phi parameters
    thetas : ndarray of shape (q,)
        The theta parameters
    mu : float
        The parameter mu
    sigma : float
        The standard deviation of the a_t random variables
    
    Returns
    -------
    log_likelihood : float
        The log-likelihood of the model
    """
    z = time_series - mu
    
    # get parameters for the sizes of everything
    p = phis.shape[0]
    q = thetas.shape[0]
    r = np.max([p,q+1])
    
    # set up necessary matrices
    bottom = np.concatenate((np.eye(r-1),np.zeros(r-1).reshape((r-1,1))),
                            axis=1) # this is the bottom of the F matrix
    F = np.concatenate((phis.reshape((1,r)),bottom),axis=0)
    H = np.concatenate(([1],thetas))
    Q = np.zeros((r,r))
    Q[0,0] = sigma**2
    
    # 
    xkp = 0
    flatPkp = la.solve(np.eye(r**2) - np.kron(F,F), Q.flatten())
    Pkp = flatPkp.reshape((r,r))
    
    loglikelihood = np.log(mvn.pdf(z[0], H.dot(xkp), H.dot(Pkp.dot(H))))
    
    for k in range(1,len(z)):
        # Update
        yk = z[k] - H.dot(xkp)
        Sk = H.dot(Pkp.dot(H.T))
        Kk = la.solve(Sk.T, H.dot(Pkp.T)).T
        xku = xkp + Kk.dot(yk)
        Pku = (np.eye(r) - Kk.dot(H)).dot(Pkp)

        # Predict
        xkp = F.dot(xku)
        Pkp = F.dot(Pku.dot(F.T)) + Q
        loglikelihood += np.log(mvn.pdf(z[k], H.dot(xkp), H.dot(Pkp.dot(H))))
    return loglikelihood

In [21]:
arma_likelihood(time_series_a, np.array([0.9]), mu=17., sigma=.4)

-2.3522949655228094